# Saint-Petersburg Bakeries

In [1]:
import pandas as pd
import geopandas as gpd
from overpy import Overpass
from shapely.geometry import Polygon
from scipy.spatial import Voronoi

from lets_plot import *
LetsPlot.setup_html()

In [2]:
def fix_bakery_name(data):
    names_fixes = {
        'Bonape': 'Bonapé',
        'Лавка пекаря': 'Лавка Пекаря',
        'Цех85': 'Цех 85',
    }
    if data['name'] in names_fixes.keys():
        data['name'] = names_fixes[data['name']]
    return data

In [3]:
def get_areas_gdf(geometry, names):
    diag = Voronoi([[p.x, p.y] for p in geometry])
    regions = [[diag.vertices[i] for i in region if i != -1] for region in diag.regions]
    regions = [Polygon(region) for region in regions if len(region) > 2]
    areas_data = [(name, next((region for region in regions if region.contains(point)), None)) \
                  for name, point in zip(names, geometry)]
    areas_df = pd.DataFrame(areas_data, columns=['name', 'area'])
    areas_gdf = gpd.GeoDataFrame(areas_df, geometry='area')
    areas_gdf = areas_gdf.set_crs(epsg=geometry.crs.to_epsg())
    areas_gdf = areas_gdf.dropna()
    return areas_gdf

We are going to use the Overpass API to get data about Saint-Petersburg's bakeries from [OSM](https://www.openstreetmap.org).

Then we will draw a Voronoi diagram of the biggest bakery chains. That will help us to find the nearest bakery for each point of the city.

In [4]:
ovp = Overpass()
spb_bakeries_data = ovp.query("""
    area[name="Санкт-Петербург"];
    nwr[shop=bakery](area);
    out center;
    """)

In [5]:
spb_bakeries_df = pd.DataFrame([(bakery.tags.get('name'), float(bakery.lon), float(bakery.lat)) \
                                for bakery in spb_bakeries_data.nodes], columns=['name', 'lon', 'lat'])
spb_bakeries_df = spb_bakeries_df.dropna()

In [6]:
spb_bakeries_gdf = gpd.GeoDataFrame(spb_bakeries_df[['name']], \
                                    geometry=gpd.points_from_xy(spb_bakeries_df.lon, spb_bakeries_df.lat))
spb_bakeries_gdf = spb_bakeries_gdf.apply(fix_bakery_name, axis=1)
top_bakery_chains = spb_bakeries_gdf['name'].value_counts()[:15].keys()
spb_bakeries_gdf = spb_bakeries_gdf[spb_bakeries_gdf['name'].isin(top_bakery_chains)]
spb_bakeries_gdf = spb_bakeries_gdf.set_crs(epsg=4326)

In [7]:
areas_gdf = get_areas_gdf(spb_bakeries_gdf.geometry.to_crs('EPSG:3857'), spb_bakeries_gdf['name']).to_crs('EPSG:4326')

In [8]:
ggplot() + \
    geom_livemap(location=[30.308611, 59.937500], zoom=11) + \
    geom_polygon(aes(group='name', fill='name'), \
                 data=areas_gdf, size=.1, color='#636363', alpha=.3, \
                 tooltips=layer_tooltips().line('@name')) + \
    geom_point(aes(group='name', fill='name'), \
               data=spb_bakeries_gdf, shape=21, color='#636363', \
               tooltips=layer_tooltips().line('@name')) + \
    scale_fill_discrete(name='chain') + \
    ggtitle('Top 15 Bakery Chains in Saint Petersburg') + \
    ggsize(800, 600)